In [9]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler  # Added for feature scaling
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler


In [10]:
def get_pca(x_train, x_test, n_components):
    pca = PCA(n_components=n_components, svd_solver='full')
    pca.fit(x_train)
    x_train = pca.transform(x_train)
    x_test = pca.transform(x_test)
    print(x_train.shape, x_test.shape)
    return x_train, x_test

In [11]:
def quantized_pso_feature_selection(csv_path, target_column=-1, n_particles=20, 
                                   max_iter=100, inertia=0.7, cognitive=1.5, 
                                   social=1.5, penalty_factor=0.01):
    """
    Quantized PSO implementation for feature selection with enhanced capabilities
    
    Parameters:
    csv_path (str): Path to CSV file
    target_column (int): Index of target column (default last)
    n_particles (int): Number of particles in swarm
    max_iter (int): Maximum iterations
    inertia (float): Inertia weight
    cognitive (float): Cognitive coefficient 
    social (float): Social coefficient
    penalty_factor (float): Feature count penalty multiplier
    
    Returns:
    tuple: (selected_features, best_score, convergence_curve)
    """
    
    # Load and preprocess data
    df = pd.read_csv(csv_path)
    # Drop rows with null values
    df_cleaned = df.dropna()
    
    # Check for null values
    null_values = df_cleaned.isnull().sum()
    
    # Display the null value counts
    print("Null value counts:")
    print(null_values)
    label_encoder = LabelEncoder()

    # Apply label encoding to categorical columns
    categorical_columns = ['id_code']
    for column in categorical_columns:
            df_cleaned[column] = label_encoder.fit_transform(df_cleaned[column])
    

    #X = df_cleaned.iloc[:, :target_column].values
    #y = df_cleaned.iloc[:, target_column].values
    X = df_cleaned.drop('label', axis=1).values  # Feature matrix
    y = df_cleaned['label'].values  # Target variable
   
    # Split data with stratification
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, stratify=y, random_state=42
    )
    
    # Feature scaling for distance-based algorithms
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    temp_train_data = X_train
    temp_test_data = X_test
    #temp_train_data, temp_test_data = get_pca(temp_train_data, temp_test_data, 0.98)
    # Initialize PSO parameters
    n_features = temp_train_data.shape[1]
    velocity_range = [-6, 6]  # Sigmoid saturation bounds
    
    # Initialize swarm positions and velocities
    particles = np.random.randint(2, size=(n_particles, n_features))
    velocities = np.random.uniform(*velocity_range, (n_particles, n_features))
    
    # Initialize personal and global bests
    personal_best = particles.copy()
    personal_scores = np.array([_evaluate(p, temp_train_data, temp_test_data, y_train, y_test, penalty_factor) for p in particles])
    global_best_idx = np.argmax(personal_scores)
    global_best = particles[global_best_idx].copy()
    global_score = personal_scores[global_best_idx]
    
    convergence = np.zeros(max_iter)
    
    # PSO optimization loop
    for iteration in range(max_iter):
        for i in range(n_particles):
            # Update velocity with clamping
            r1, r2 = np.random.rand(2, n_features)
            velocities[i] = (inertia * velocities[i] +
                            cognitive * r1 * (personal_best[i] - particles[i]) +
                            social * r2 * (global_best - particles[i]))
            
            # Apply velocity clamping
            velocities[i] = np.clip(velocities[i], *velocity_range)
            
            # Update position using sigmoid probability
            prob = 1 / (1 + np.exp(-velocities[i]))
            particles[i] = (np.random.rand(n_features) < prob).astype(int)
            
            # Evaluate and update bests
            current_score = _evaluate(particles[i],temp_train_data, temp_test_data, y_train, y_test, penalty_factor)
            if current_score > personal_scores[i]:
                personal_best[i] = particles[i].copy()
                personal_scores[i] = current_score
                
                if current_score > global_score:
                    global_best = particles[i].copy()
                    global_score = current_score
        
        convergence[iteration] = global_score
        print(f"Iter {iteration+1}/{max_iter} | Best Score: {global_score:.4f}")
    
    # Final feature selection
    selected_features = np.where(global_best == 1)[0]
    return selected_features, global_score, convergence


In [12]:
def _evaluate(particle, X_train, X_test, y_train, y_test, penalty_factor):
    """Enhanced evaluation function with cross-validation"""
    selected = particle.astype(bool)
    if not np.any(selected):
        return 0
    
    # Reduced feature subset
    X_train_sub = X_train[:, selected]
    X_test_sub = X_test[:, selected]
    
    # Classifier with cross-validation
    clf = KNeighborsClassifier(n_neighbors=5)
    clf.fit(X_train_sub, y_train)
    
    #clf=SVC(C = 19,kernel='linear')
    #clf.fit(X_train_sub, y_train)
    acc = accuracy_score(y_test, clf.predict(X_test_sub))
    
    # Penalty term calculation
    penalty = penalty_factor * (np.sum(particle) / particle.size)
    return acc - penalty

In [19]:
# Example usage
if __name__ == "__main__":
    #file_path = 'featureoptimization/featureoptimization/densenet201_lbp_histog_feats.csv'
    #file_path = 'featureoptimization/featureoptimization/densenet201feat.csv'
    file_path = 'featureoptimization/featureoptimization/efficientnetb3newfeat.csv'
    #file_path = 'featureoptimization/featureoptimization/combined_DIEnet_features.csv'
    #file_path = 'featureoptimization/featureoptimization/inceptionv4newfeat.csv'
    features, score, curve = quantized_pso_feature_selection(
        
        file_path,
        n_particles=30,
        max_iter=10,
        penalty_factor=0.015
    )
    print(f"Selected features: {features}")
    print(f"Validation score: {score:.4f}")

Null value counts:
id_code               0
effb3_feature_0       0
effb3_feature_1       0
effb3_feature_2       0
effb3_feature_3       0
                     ..
effb3_feature_1532    0
effb3_feature_1533    0
effb3_feature_1534    0
effb3_feature_1535    0
label                 0
Length: 1538, dtype: int64
Iter 1/10 | Best Score: 0.4027
Iter 2/10 | Best Score: 0.4027
Iter 3/10 | Best Score: 0.4027
Iter 4/10 | Best Score: 0.4027
Iter 5/10 | Best Score: 0.4027
Iter 6/10 | Best Score: 0.4027
Iter 7/10 | Best Score: 0.4027
Iter 8/10 | Best Score: 0.4034
Iter 9/10 | Best Score: 0.4045
Iter 10/10 | Best Score: 0.4047
Selected features: [   0    1    2    4    6    7    8    9   10   11   12   13   14   15
   20   21   22   23   24   25   30   31   32   33   36   41   47   49
   52   54   58   66   68   70   72   77   80   81   84   88   89   95
   99  101  102  103  104  106  107  109  110  112  113  114  117  118
  119  121  123  128  129  130  131  132  135  136  138  139  140  141
  142

In [14]:
len(features)

1222

In [20]:
df = pd.read_csv('featureoptimization/featureoptimization/efficientnetb3newfeat.csv')
df = df[df.columns[1:]]
df.isnull().sum().max()
#understanding the predicted value - which is hot encoded, in real life price won't be hot encoded.
df['label'].describe(), df['label'].unique()

# there are 4 classes in the predicted value
y_t = np.array(df['label'])
X_t = df = df.iloc[:, features]
#X_t = df.drop(['label'],axis=1)
X_t = np.array(X_t)
print("shape of Y :"+str(y_t.shape))
print("shape of X :"+str(X_t.shape))
scaler = MinMaxScaler()
X_t = scaler.fit_transform(X_t)

shape of Y :(16925,)
shape of X :(16925, 767)


In [21]:
X_train,X_test,Y_train,Y_test = train_test_split(X_t,y_t,test_size=.20,random_state=42)
print("shape of X Train :"+str(X_train.shape))
print("shape of X Test :"+str(X_test.shape))
print("shape of Y Train :"+str(Y_train.shape))
print("shape of Y Test :"+str(Y_test.shape))

shape of X Train :(13540, 767)
shape of X Test :(3385, 767)
shape of Y Train :(13540,)
shape of Y Test :(3385,)


In [49]:
#temp_train_data, temp_test_data = get_pca(X_train, X_test, 0.98)

(13540, 646) (3385, 646)


In [ ]:
for this_C in [1,2,3,4,5,6,7,8,9,10,11,12,13,17,18,19,20]:
    svm_ova = SVC(C = this_C,kernel='rbf', decision_function_shape='ovr')
    svm_ova.fit(X_train, Y_train)
    y_pred_ova = svm_ova.predict(X_test)
    print("One-vs-All Accuracy:", accuracy_score(Y_test, y_pred_ova))

One-vs-All Accuracy: 0.43751846381093057
